<a href="https://colab.research.google.com/github/Lrraymond13/collab/blob/master/LRR_MIT6_036_hw08_colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MIT 6.036 Spring 2019: Homework 8#

This colab notebook provides code and a framework for [homework 8](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week8/week8_homework/).  You can work out your solutions here, then submit your results back on the homework page when ready.

## <section>**Setup**</section>

First, download the code distribution for this homework that contains test cases and helper functions.

Run the next code block to download and import the code for this lab.


In [0]:
!rm -rf code_for_hw8* __MACOSX data .DS_Store
!wget --quiet https://introml.odl.mit.edu/cat-soop/_static/6.036/homework/hw08/code_for_hw8.zip
!unzip code_for_hw8.zip
!mv code_for_hw8/* .

from code_for_hw8_oop import *
import numpy as np
import math as m

In [0]:
# set random seed for question 3g
 np.random.seed(0)

# 1) Batch Normalization
No code involved,  <a href="https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week8/week8_homework/">Refer to Catsoop.</a>

# 2) Implementing Mini-batch Gradient Descent and Batch Normalization (OPTIONAL)

** Note: You can click the arrow on the left of this text block to collapse/expand this optional section and all its code blocks **

Last week we implemented a framework for building neural networks from scratch. We trained our models using *stochastic* gradient descent. In this problem, we explore how we can implement batch normalization as a module `BatchNorm` in our framework. It is the same module which you analyzed in problem 1. 

Key to the concept of batch normalization is the doing gradient descent on batches of data. So we instead of using last week's stochastic gradient descent, we will first implement the *mini-batch* gradient descent method `mini_gd`, which is a hybrid between *stochastic* gradient descent and *batch* gradient descent. The lecture notes on <a href="https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week7/neural_networks_2/1?activate_block_id=block-v1%3AMITx%2B6.036%2B2019_Spring%2Btype%40vertical%2Bblock%40neural_networks_2_optimizing_neural_network_parameters_vert"> optimizing neural network parameters</a> are helpful for this part.

In *mini-batch* gradient descent, for a mini-batch of size $K$, we select $K$ distinct data points uniformly at random from the data set and update the network weights based only on their contributions to the gradient:
$$W := W - \eta\sum_{i=1}^K \nabla_W \mathcal{L}(h(x^{(i)}; W), y^{(i)})\;\;.$$

Our *mini-batch* method `mini_gd` will be implemented within the `Sequential` python class (see homework 7 problem 2) and will take the following as inputs:

* `X`: a standard data array (d by n)
* `y`: a standard labels row vector (1 by n)
* `iters`: the number of updates to perform on weights $W$
* `lrate`: the learning rate used
* `K`: the mini-batch size to be used

One call of `mini_gd` should call `Sequential.backward` for back-propagation and `Sequential.step` for updating the weights, for a total of `iters` times, using `lrate` as the learning rate. As in our implementation of `sgd` from homework 7, we compute the predicted output for a mini-batch of data with the `Sequential.forward` method. We compute the loss between our predictions and the true labels using the assigned `Sequential.loss` method. (Note that in homework 7, `Sequential.step` was called `Sequential.sgd_step`. While the functionality of the step function is the same, it has been renamed for convenience. The same is true for the `module.step` function of each module we implemented, where applicable.)

For picking $K$ unique data points at random from our large data-set for each mini-batch, we will implement the following strategy: we will first shuffle our data points `X` (and associated labels `y`). Then, we get <math>\frac{n}{k}</math> (rounded down to the nearest integer) different mini-batches by grouping each $K$ consecutive points from this shuffled array. If we end up iterating over all the points but need more mini-batches, we will repeat the shuffling and the batching process.

<b>2A)</b>You need to fill in the missing code below. We have implemented the shuffling of indices and have provided you with the outer and inner loops.

** This OPTIONAL problem has you extend your homework 7 implementation for building neural networks. **
### PLEASE COPY IN YOUR CODE FROM HOMEWORK 7 TO COMPLEMENT THE CLASSES GIVEN HERE

Recall that your implementation from homework 7 included the following classes:
    
  * Module
  * Linear 
  * Tanh 
  * ReLU 
  * SoftMax
  * NLL
  * Sequential

In [0]:
import math as m 

class Sequential:
    def __init__(self, modules, loss):            
        self.modules = modules
        self.loss = loss

    def mini_gd(self, X, Y, iters, lrate, notif_each=None, K=10):
        D, N = X.shape

        np.random.seed(0)
        num_updates = 0
        indices = np.arange(N)
        while num_updates < iters:

            np.random.shuffle(indices)
            X = None  # Your code
            Y = None  # Your code

            for j in range(m.floor(N/K)):
                if num_updates >= iters: break

                # Implement the main part of mini_gd here
                # Your code
                
                num_updates += 1

    def forward(self, Xt):                        
        for m in self.modules: Xt = m.forward(Xt)
        return Xt

    def backward(self, delta):                   
        for m in self.modules[::-1]: delta = m.backward(delta)

    def step(self, lrate):    
        for m in self.modules: m.step(lrate)

<b>2B)</b> We are now ready to implement batch normalization into our neural network framework! Our module `BatchNorm` will sit between consecutive layers of neurons, such as the $l^{th}$ and $(l+1)^{th}$ layers, acting as a "corrector" which allows $W^l$ to change freely, producing outputs $z^l$, but then the module corrects the covariate shift induced in the signals before they reach the $(l+1)^{th}$ layer, converting $z^l$ to $\widehat{Z}^l$. 

The following is a summmary what is described in the <a href="https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week7/neural_networks_2/2">lecture notes</a>, and it should guide your implementation of the module. 

Any normalization between the $l^{th}$ and $(l+1)^{th}$ layers is done *separately* for each of the $n^l$ input connections leading to the $(l+1)^{th}$ layer. We handle a mini-batch of data of size $K$, and $Z^l$ is $n^l \times K$, and the output $\widehat{Z}^l$is of the same shape. 

We first compute $n^l$ *batchwise* means and
standard deviations.  Let $\mu^l$ be the $n^l \times 1$ vector (`self.mus`) where
$$\mu^l_i = \frac{1}{K} \sum_{j = 1}^K Z^l_{ij}\;\;,$$
and let $\sigma^l$ be the $n^l \times 1$ vector (`self.vars`) where 
$$\sigma^l_i = \sqrt{\frac{1}{K} \sum_{j = 1}^K (Z^l_{ij} - \mu_i)^2}\;\;.$$

The normalized data `self.norm` is the matrix $\overline{Z}$, where
$$\overline{Z}^l_{ij} = \frac{Z^l_{ij} - \mu^l_i}{\sigma^l_i + \epsilon}\;\;,$$
and where $\epsilon$ is a very small constant to guard against division by
zero. 

We define weights $G^l$ (`self.G`) and $B^l$ (`self.B`), each being an $n^l \times 1$ vector, which we use to to shift and scale the outputs:
$$\widehat{Z}^l_{ij} = G^l_i \overline{Z}^l_{ij} + B^l_i\;\;.$$

The outputs are finally ready to be passed to the $(l+1)^{th}$ layer.

A slight warning (that we will not worry about here) about `BatchNorm` is that during the *test* phase, if the test mini-batch size is too small (imagine we are deploying a neural network that deals with live video frames), then the lack of samples would cause the freshly-calculated $\mu^l$ and $\sigma^l$ to be far off from their true values that the module's parameters $G^l$ and $B^l$ were trained to be compatible with. To fix that, people usually compute a running average of $\mu^l$ and $\sigma^l$ during training, to be used at test time. We will assume our test mini-batches are large enough.

In this problem we only implement the `BatchNorm.forward` and `BatchNorm.step` methods. We provide you with the implementation for `BatchNorm.backward` and the lecture notes contain the details of the derivations. You will need to fill in the missing code below.

In [0]:
class BatchNorm(Module):    
    def __init__(self, m):
        np.random.seed(0)
        self.eps = 1e-20
        self.m = m  # number of input channels
        
        # Init learned shifts and scaling factors
        self.B = np.zeros([self.m, 1])
        self.G = np.random.normal(0, 1.0 * self.m ** (-.5), [self.m, 1])
        
    # Works on m x b matrices of m input channels and b different inputs
    def forward(self, A):# A is m x K: m input channels and mini-batch size K
        # Store last inputs and K for next backward() call
        self.A = A
        self.K = A.shape[1]
        
        self.mus = None  # Your Code
        self.vars = None  # Your Code

        # Normalize inputs using their mean and standard deviation
        self.norm = None  # Your Code
            
        # Return scaled and shifted versions of self.norm
        return None  # Your Code

    def backward(self, dLdZ):
        # Re-usable constants
        std_inv = 1/np.sqrt(self.vars+self.eps)
        A_min_mu = self.A-self.mus
        
        dLdnorm = dLdZ * self.G
        dLdVar = np.sum(dLdnorm * A_min_mu * -0.5 * std_inv**3, axis=1, keepdims=True)
        dLdMu = np.sum(dLdnorm*(-std_inv), axis=1, keepdims=True) + dLdVar * (-2/self.K) * np.sum(A_min_mu, axis=1, keepdims=True)
        dLdX = (dLdnorm * std_inv) + (dLdVar * (2/self.K) * A_min_mu) + (dLdMu/self.K)
        
        self.dLdB = np.sum(dLdZ, axis=1, keepdims=True)
        self.dLdG = np.sum(dLdZ * self.norm, axis=1, keepdims=True)
        return dLdX

    def step(self, lrate):
        self.B = None  # Your Code
        self.G = None  # Your Code
        return

# 3) 2D Datasets

For the 2D datasets, we have provided the following function:


```
run_keras_2d(data_name, layers, epochs, split=0.25, display=True, trials=5)
```


where:

data_name is a string, such as '1', '2', etc.
layers is a list of Keras layer definitions for a Sequential model, e.g. 
```
[Dense(input_dim=2, units=10, activation='relu'), Dense(units=2, activation='softmax')]
```

epochs is an integer indicating how many times to go through the data in training
split is a fraction of the training data to use for validation if a validation set is not defined
display whether to display result plots
verbose whether to print loss and accuracy (percent correctly labeled) each epoch
trials is an integer indicating how many times to perform the training and testing
2D Data
The two-class datasets have data_names: '1','2','3','4'. Target accuracies (percent correct) on the validation set are (99%, 90.5%, 96%, 94%).

In this problem, try the following 5 architectures, specified by the number of units in the hidden layers:

1: (0), 2: (10), 3: (100), 4: (10, 10), 5: (100, 100))
You may find the archs function in the code file to be helpful here.
Some of these questions ask for the "simplest" architecture; the list above is ordered starting with the simplest.

In [0]:
from code_for_hw8_keras import *
import keras
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.layers import Conv1D, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import Callback
from keras.datasets import mnist
from keras import backend as K
from keras.initializers import VarianceScaling
from matplotlib import pyplot as plt 

#]def archs(classes):
    return [[Dense(input_dim=2, units=classes, activation="softmax")],
            [Dense(input_dim=2, units=10, activation='relu'),
             Dense(units=classes, activation="softmax")],
            [Dense(input_dim=2, units=100, activation='relu'),
             Dense(units=classes, activation="softmax")],
            [Dense(input_dim=2, units=10, activation='relu'),
             Dense(units=10, activation='relu'),
             Dense(units=classes, activation="softmax")],
            [Dense(input_dim=2, units=100, activation='relu'),
             Dense(units=100, activation='relu'),
             Dense(units=classes, activation="softmax")]]

In [0]:
# 3A answer
run_keras_2d("1", archs(2)[0], 10, display=False, verbose=False, trials=5)
run_keras_2d("2", archs(2)[4], 10, display=False, verbose=False, trials=5)
run_keras_2d("3", archs(2)[1], 10, display=False, verbose=False, trials=5)
run_keras_2d("4", archs(2)[1], 10, display=False, verbose=False, trials=5)

In [0]:
# 3C 
run_keras_2d("3", [
    Dense(input_dim=2, units=200, activation="relu"),
    Dense(units=200, activation="relu"),
    Dense(units=2, activation="softmax")
], 100)

In [0]:
# 3E average validation accuracy
run_keras_2d("3class", archs(3)[0], 10, split=0.5, display=False, verbose=False, trials=5)
run_keras_2d("3class", archs(3)[1], 10, split=0.5, display=False, verbose=False, trials=5)
run_keras_2d("3class", archs(3)[2], 10, split=0.5, display=False, verbose=False, trials=5)
run_keras_2d("3class", archs(3)[3], 10, split=0.5, display=False, verbose=False, trials=5)
run_keras_2d("3class", archs(3)[4], 10, split=0.5, display=False, verbose=False, trials=5)

In [0]:
#example of run_keras_2d
# this is a 2 class dataset

# for dataset in ["1", "2", "3", "4"]:
for dataset in ["3class"]:
  for archnum in rAange(5):
    print('Dataset ', dataset, " Architecture ", archnum)
    run_keras_2d(dataset, archs(3)[archnum], 10, split=0.5, display=False, verbose=False, trials=5)




In [0]:
# USE THE 3 class dataset and run for no hidden units with 1 trial
linear_model = archs(3)[0]
run_keras_2d('3class', linear_model, 10, split=0.5, display=False, verbose=False, trials=1)



In [0]:
# GET weights from the first layer of the model 
w, w0 = linear_model[0].get_weights()

In [0]:
# compute z values for each of the following points
X = np.array([[-1,0], [1,0], [0,-11], [0,1], [-1,-1], [-1,1], [1,1], [1,-1]])

In [0]:
z = np.dot(w.T, X.T) + w0.reshape((-1,1))

In [0]:
# find indices with argmax
np.argmax(z, axis=0)

# Weight sharing (OPTIONAL)

** Note: You can click the arrow on the left of this text block to collapse/expand this optional section and all its code blocks **

In the lab we designed a CNN that can count the number of objects in 1 dimensional images, where each black pixel is represented by a value of 0 and each white pixel is represented by a value of 1. Recall that an object is a consecutive sequence of black pixels ($0$'s). For example, the sequence $0100110$ contains three objects. 

Here we want to see how hard/easy it is to train such a network from data.  Our network architecture will be as follows: 

* The first layer is convolutional and you will implement it using the Keras `Conv1D` function, with a kernel of size 2 and stride of 1 with ReLu activation.

* The second layer is a fully connected `Dense` layer which has a scalar output. 

Here is sample usage of the `Conv1D` and`Dense` layers. 

`layer1=keras.layers.Conv1D(filters=?, kernel_size=?, strides=?,use_bias=False, activation=?, batch_size=1, input_shape=?, padding='same')`

`Dense(units=?, activation=?, use_bias=False)`

You need to fill in the parameters marked with `?` based on the problem specifications. Note also that in Keras, depending on your implementation, you may be forced to use *three* layers to implement such a network, where one intermediary `Flatten` layer is used to flatten the output of the convolutional layer, before being passed to the dense layer.

Refer to the <a href="https://keras.io/layers/convolutional/">Conv 1D</a>, <a href="https://keras.io/layers/core/">Dense</a> and <a href="https://keras.io/layers/core/#flatten">Flatten</a> descriptions in the Keras documentation to see the available parameter options.

In this exercise, we fix the structure and want to learn the best combination of weights from data. In the homework code, we have provided functions `train_neural_counter` and `get_image_data_1d`. You can use them to generate data and train the above neural network in Keras to answer the following questions. We assume that the images in our data set are randomly generated. The probability of a pixel being white is $0.1$. We work with mean squared error as the loss function for this problem. We have provided template code which you can fill in, to perform the training.

<b>4A)</b> We call a $2\times 1$ filter an *edge detector* if it has weights $[-1,1]$ or $[1,-1]$ (i.e., it can detect a left or right edge in a given image). We call it an *averaging filter* if takes average of nearby pixels. 

Assuming that the second layer in our network is the sum function (i.e., all of its weights are equal to 1), which of the following filters gives a (nearly) local minima? You may assume that we are working with $1024\times 1$ images. 

(Although you can answer the question without any coding, you may also empirically verify your claims. The function `set_weights` that can be applied to keras layers might be useful.)

<b>4B)</b> What is (approximately) the expected loss of the network on $1024\times 1$ images if the convolutional layer is an averaging filter and second layer is the sum function (without a bias term)? (Note that you can answer the question theoretically or through coding, depending on your preference.)

In [0]:
from keras.models import Sequential

In [0]:
# Code template if you would like to check 4B) through code

imsize = 1024
prob_white = 0.1

num_filters = None  # Your code
kernel_size = None  # Your code
strides = None  # Your code
activation_conv = None  # Your code

(X_train,Y_train,X_val,Y_val,X_test,Y_test) = get_image_data_1d(1000,imsize,prob_white)

layer1=keras.layers.Conv1D(filters=num_filters, kernel_size=kernel_size, \
       strides=strides, use_bias=False, activation=activation_conv, batch_size=1, input_shape=(imsize,1), padding='same')

activation_dense = None  # Your code
num_units = None  # Your code
layer3=Dense(units=num_units, activation=activation_dense, use_bias=False)

layers=[layer1,Flatten(),layer3]

# This is how we create the model using our layers
model=Sequential()
for layer in layers:
    model.add(layer)
        
model.compile(loss='mse', optimizer=Adam()) 

# Set the weights of the layers to desired values
# We give you the lines to use for this part
model.layers[0].set_weights([np.array([1/2,1/2]).reshape(2,1,1)])
model.layers[-1].set_weights([np.ones(imsize).reshape(imsize,1)])

model.evaluate(X_test,Y_test)

<b>4C)</b> Now suppose we add a bias term of $-10$ to the last layer. What is (approximately) the expected quadratic loss? (Note that you can answer the question theoretically or through coding, depending on your preference.)

In [0]:
# Edit code from 4B) with the bias

<b>4D)</b> Averaging type filters are abundant and form a nearly flat valley of local minima for this problem. It is difficult for the network to find alternative solutions on its own. We need to force our way out of these bad minima and towards a better solution, i.e., an edge detector. To force the first layer to behave as an edge detector, we need to choose a proper **kernel regularizer**. Consider the following functions

$f_1=\sum_i |w_i|$, $f_2=\sum_i |w_i^2|$, $f_3=|\sum_{i} w_i|$. Which one of the choices is likely to guide the network to find an edge detector at the convolution layer?


<a href="https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week8/week8_homework/">Refer to Catsoop.</a>

Implement your choice of regularizers from above in the code (complete the function `filter_reg`). Do not allow any bias in the layers for the rest of the problem. The code generates some random test and training data sets and trains the model on these data. Run a few learning trials (5 or more) for each data set and answer the following questions based on the performance of your model.

**IMPORTANT**: When implementing `filter_reg`, you should use the keras backend operations, imported as "K" in the code. So for example, `K.sum` and `K.abs`, rather than `np.sum` and `np.abs`. This is because the `weights` argument is NOT a numpy object, but rather an internal Keras object!

In [0]:
# Implement filter_reg

def filter_reg(weights):
    # We scale the output of the filter by lam
    lam=1000
    filter_result = None  # Your code
    return lam * filter_result

<b>4E)</b> For $1024\times 1$ images and training set of size $1000$, is the network **without any regularization** likely to find models that have a mean square error lower than 8 on the test data?

In [0]:
# Code template if you would like to check 4B) through code

imsize = 1024
prob_white = 0.1

data=get_image_data_1d(1000, imsize, prob_white)
trials=5
for trial in range(trials):
  
    num_filters = None  # Your code
    kernel_size = None  # Your code
    strides = None  # Your code
    activation_conv = None  # Your code

    layer1=keras.layers.Conv1D(filters=num_filters, kernel_size=kernel_size, \
    strides=strides, use_bias=False, activation=activation_conv, batch_size=1, \
    input_shape=(imsize,1),padding='same')
    
    activation_dense = None  # Your code
    num_units = None  # Your code

    layer3=Dense(units=num_units, activation=activation_dense, use_bias=False)
    
    layers=[layer1,Flatten(),layer3]
    model,err = train_neural_counter(layers, data, 'mse')
    
    model.layers[0].get_weights()[0]
    np.mean(model.layers[-1].get_weights()[0])
    print(err)

#### For parts F) to J), simply edit your code from E) with the necessary changes.

<b>4F)</b> Repeat the same experiment, but now with the regularizer you implemented. Try different regularization parameters. Which choice of regularization parameter gives the best prediction results?

In [0]:
# Edit code from 4E), using your filter as the kernel_regularizer in the Conv1D layer

<b>4G)</b> With the above choice of regularization parameter, what is the mean square error of the best network that you find on the test data? Try a few trials (5 or more) for each data test and report the value of the best network. 



#### We expect the training to be easier when there are fewer parameters to learn. Consider images of size $128\times 1$ for the rest of the problem.

<b>4H)</b> Instead of resorting to regularization again, we may instead find a way to reduce the number of parameters. What additional layer can you add to the output of the convolution layer to reduce the number of parameters to be learned without losing any relevant information?

<a href="https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week8/week8_homework/">Refer to Catsoop.</a>

<b>4I)</b> Add the layer you suggested above to your network and run some tests with data sets of size 1000 on $128\times 1$ images.  How many parameters are left to learn with the new structure?

<b>4J)</b> Mark your observations on the two structures (not using regularization).
<a href="https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week8/week8_homework/">Refer to Catsoop.</a>

# 5) MNIST (Digit Classification)

In this section, we'll be looking at the MNIST data set seen already in problem 2. This time, we look at the *complete* MNIST problem where our networks will take an image of *any* digit from $0-9$ as input (recall that problem 2 only looked at digits $0$ and $1$) and try to predict that digit. Also, we will now use out-of-the-box neural network implementations using Keras and Tensorflow. State-of-the-art systems have error rates of less that one half of one percent on this data set (see <a href="http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#4d4e495354">this list</a>).  We'll be happy with an error rate less than 2% since we don't have all year...
<br>

You can access the MNIST data for this problem using:
<br><code>train, validation = get_MNIST_data()</code>
<br>

You can run the fully connected MNIST model, using:
<br><code>run_keras_fc_mnist(train, validation, layers, epochs, split=0.1, trials=5)</code>
<br>

And, you can run the CNN MNIST test, using:
<br><code>run_keras_cnn_mnist(train, validation, layers, epochs, split=0.1, trials=5)</code>
<br>

For all following experiments, please run for 5 trials (use `trials=5`) and report the average accuracy.
<br>

A word of warning, if you have a machine with a single core and/or very little RAM, you'll be better off running on an Athena workstation. If your solutions are not being accepted, and you are confident in your approach, try with more trials. Also, 
<br>

You will need to design your own `layers` to feed to `run_keras_fc_mnist` and `run_keras_cnn_mnist`, which will be different than the ones specified by `archs()`. For instance, `layers=[Dense(input_dim=64, units=4, activation="softmax")]` defines a single layer with 64 inputs, 4 output units, and softmax activation. Also, we advise you to use the option `verbose=True` when unsure about the progress made during training of your models.
<br>

<b> 5A)</b> Look at the code and indicate what the difference is between <code>run_keras_fc_mnist</code> and <code>run_keras_cnn_mnist</code>? <a href="https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week8/week8_homework/">Refer to Catsoop.</a>

In [0]:
def run_keras_fc_mnist(train, test, layers, epochs, split=0.1, verbose=True, trials=1):
    (X_train, y1), (X_val, y2) = train, test
    # Flatten the images
    m = X_train.shape[1]
    X_train = X_train.reshape((X_train.shape[0], m*m))
    X_val = X_val.reshape((X_val.shape[0], m*m))
    # Categorize the labels
    num_classes = 10
    y_train = np_utils.to_categorical(y1, num_classes)
    y_val = np_utils.to_categorical(y2, num_classes)
    # Train, use split for validation
    val_acc, test_acc = 0, 0
    for trial in range(trials):
        # Reset the weights
        # See https://github.com/keras-team/keras/issues/341
        session = K.get_session()
        for layer in layers:
            for v in layer.__dict__:
                v_arg = getattr(layer, v)
                if hasattr(v_arg, 'initializer'):
                    initializer_func = getattr(v_arg, 'initializer')
                    initializer_func.run(session=session)
        # Run the model
        model, history, vacc, tacc = \
                run_keras(X_train, y_train, X_val, y_val, None, None, layers, epochs, split=split, verbose=verbose)
        val_acc += vacc if vacc else 0
        test_acc += tacc if tacc else 0
    if val_acc:
        print ("\nAvg. validation accuracy:"  + str(val_acc/trials))
    if test_acc:
        print ("\nAvg. test accuracy:"  + str(test_acc/trials))

def run_keras_cnn_mnist(train, test, layers, epochs, split=0.1, verbose=True, trials=1):
    # Load the dataset
    (X_train, y1), (X_val, y2) = train, test
    # Add a final dimension indicating the number of channels (only 1 here)
    m = X_train.shape[1]
    X_train = X_train.reshape((X_train.shape[0], m, m, 1))
    X_val = X_val.reshape((X_val.shape[0], m, m, 1))
    # Categorize the labels
    num_classes = 10
    y_train = np_utils.to_categorical(y1, num_classes)
    y_val = np_utils.to_categorical(y2, num_classes)
    # Train, use split for validation
    val_acc, test_acc = 0, 0
    for trial in range(trials):
        # Reset the weights
        # See https://github.com/keras-team/keras/issues/341
        session = K.get_session()
        for layer in layers:
            for v in layer.__dict__:
                v_arg = getattr(layer, v)
                if hasattr(v_arg, 'initializer'):
                    initializer_func = getattr(v_arg, 'initializer')
                    initializer_func.run(session=session)
        # Run the model
        model, history, vacc, tacc = \
                run_keras(X_train, y_train, X_val, y_val, None, None, layers, epochs, split=split, verbose=verbose)
        val_acc += vacc if vacc else 0
        test_acc += tacc if tacc else 0
    if val_acc:
        print ("\nAvg. validation accuracy:"  + str(val_acc/trials))
    if test_acc:
        print ("\nAvg. test accuracy:"  + str(test_acc/trials))

<b> 5B)</b> Using one epoch of training, what is the accuracy of a network **with no hidden units** (using the <code>run_keras_fc_mnist</code> method) on this data? Hint: this is expected to be terrible. If it's still not working, run for more trials. Remember to use 10 output units (the network predicts a digit from 0-9) and softmax activation!



In [0]:
# get mnist data
train, validation = get_MNIST_data()

<b> 5C)</b> When creating the keras layer, pass in the following argument to Dense:
<code>kernel_initializer=VarianceScaling(scale=0.001, mode='fan_in', distribution='normal', seed=None)</code> and repeat the test.  What is the accuracy now?


In [0]:
# fully connected flattens the 28*28 image so needs to have 28^2 numbers of units
# 10 possible outputs (1 out of ten digits)
layers = [Dense(input_dim=28*28, units=10, activation="softmax")] # Your code
layers_initialize = [Dense(input_dim=28*28, units=10, activation="softmax", 
                          kernel_initializer=VarianceScaling(scale=0.001, mode='fan_in', distribution='normal', seed=None))] # Your code
# this is a way of initializing the weights to adapt to scale of the weights
#With distribution="normal", samples are drawn from a truncated normal distribution centered on zero, with stddev = sqrt(scale / n) where n is:

   # number of input units in the weight tensor, if mode = "fan_in"
  #  number of output units, if mode = "fan_out"
  #  average of the numbers of input and output units, if mode = "fan_avg"
# seems to boost performance

# suppose I linearly scale the data so pixel values are between 0 and 1, repeat test without variance scaling
# each grayscale pixel is between 0 and 255 
# to do this divide each by 255

train_scaled  = train[0]/255.0
validation_scaled  = validation[0]/255.0

<b> 5D)</b> Now, linearly scale the data so that the pixel values are between 0 and 1 and repeat your test with the original layer (no VarianceScaling). What is the accuracy now?

In [25]:
run_keras_fc_mnist(
    (train_scaled, train[1]), (validation_scaled, validation[1]), layers, epochs=1, split=0.1, verbose=True, trials=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 3s 58us/step - loss: 0.4713 - acc: 0.8758 - val_loss: 0.3076 - val_acc: 0.9155

Loss on validation set:0.3075888371527195 Accuracy on validation set: 0.9155
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 4s 59us/step - loss: 0.4683 - acc: 0.8769 - val_loss: 0.3063 - val_acc: 0.9172

Loss on validation set:0.3062611770927906 Accuracy on validation set: 0.9172
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 4s 59us/step - loss: 0.4711 - acc: 0.8751 - val_loss: 0.3104 - val_acc: 0.9153

Loss on validation set:0.3104010093510151 Accuracy on validation set: 0.9153
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 4s 61us/step - loss: 0.4658 - acc: 0.8788 - val_loss: 0.3075 - val_acc: 0.9160

Loss on validation se

In [0]:
# check size of images
len(train)

In [0]:
layers = None  # Your code
train, validation = get_MNIST_data()

# Scale the images
train = (None, train[1])  # Your code
validation = (None, validation[1])  # Your code

run_keras_fc_mnist(train, validation, layers, 1, split=0.1, verbose=True, trials=5)

<b> 5E)</b> What is happening? <a href="https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week8/week8_homework/">Refer to Catsoop.</a>


### Important: <b>Always scale the data like in 5D) for subsequent problems.</b>


<b> 5F)</b> Using this same architecture, evaluate validation accuracy for number of training epochs in [5, 10, 15].

In [26]:
train, validation = get_MNIST_data()

# Scale the images
train = (train[0]/255.0, train[1])  # Your code
validation = (validation[0]/255.0, validation[1])  # Your code

for epochs in [5,10,15]:
    layers = [Dense(input_dim=28*28, units=10, activation="softmax")] # Your code
    run_keras_fc_mnist(train, validation, layers, epochs, split=0.1, verbose=True, trials=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 63us/step - loss: 0.4765 - acc: 0.8750 - val_loss: 0.3097 - val_acc: 0.9145
Epoch 2/5
60000/60000 [==============================] - 3s 54us/step - loss: 0.3047 - acc: 0.9153 - val_loss: 0.2809 - val_acc: 0.9218
Epoch 3/5
60000/60000 [==============================] - 3s 50us/step - loss: 0.2831 - acc: 0.9216 - val_loss: 0.2733 - val_acc: 0.9231
Epoch 4/5
60000/60000 [==============================] - 3s 50us/step - loss: 0.2731 - acc: 0.9237 - val_loss: 0.2714 - val_acc: 0.9228
Epoch 5/5
60000/60000 [==============================] - 3s 49us/step - loss: 0.2666 - acc: 0.9253 - val_loss: 0.2689 - val_acc: 0.9240

Loss on validation set:0.2688548969298601 Accuracy on validation set: 0.924
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 62us/step - loss: 0.4680 - acc: 0.8785 - val_loss: 0.3092 - val_acc: 0.9137
Epoch 2

<b> 5G)</b> 

<b> 5G a)</b>With the validation accuracy that you just saw on per digit basis using $15$ epochs, and assuming each digit is read independently from the others, what is the probability of reading a 5 digit zip code correctly?<br>

<b> 5G b)</b>Now, assume that the accuracy is 0.9985, what is the probability of reading a zip code correctly?<br>
b.

This is why people care about dropping the error rates to what at first sound like ridiculous values.

<a href="https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week8/week8_homework/">Refer to Catsoop.</a>

<b> 5H)</b> Using one epoch of training, try a single hidden layer with ReLU and gradually increase the units (128, 256, 512, 1024) units.  What are the accuracies?

In [28]:

for num in [128,256,512,1024]:
    layers =  [Dense(input_dim=28*28, units=num, activation='relu'), Dense(units=10, activation="softmax")] # Your code
    run_keras_fc_mnist(train, validation, layers, 1, split=0.1, verbose=True, trials=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 10s 164us/step - loss: 0.2531 - acc: 0.9280 - val_loss: 0.1321 - val_acc: 0.9612

Loss on validation set:0.13212243243679403 Accuracy on validation set: 0.9612
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 9s 153us/step - loss: 0.2529 - acc: 0.9279 - val_loss: 0.1336 - val_acc: 0.9620

Loss on validation set:0.13358383609205485 Accuracy on validation set: 0.962
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 8s 140us/step - loss: 0.2566 - acc: 0.9271 - val_loss: 0.1374 - val_acc: 0.9598

Loss on validation set:0.1373662525974214 Accuracy on validation set: 0.9598
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 9s 144us/step - loss: 0.2629 - acc: 0.9243 - val_loss: 0.1395 - val_acc: 0.9596

Loss on validat

<b> 5I)</b> Now, try a network with two layers, again using one epoch, with 512 units in the first hidden layer and and 256 units in the second hidden layer.  What is the accuracy?

In [29]:
layers =  [Dense(input_dim=28*28, units=512, activation='relu'), 
           Dense(input_dim=28*28, units=256, activation='relu'), 
           Dense(units=10, activation="softmax")] # Your code
run_keras_fc_mnist(train, validation, layers, 1, split=0.1, verbose=True, trials=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 21s 354us/step - loss: 0.1852 - acc: 0.9431 - val_loss: 0.1022 - val_acc: 0.9674

Loss on validation set:0.10222592464890332 Accuracy on validation set: 0.9674
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 21s 349us/step - loss: 0.1852 - acc: 0.9445 - val_loss: 0.0988 - val_acc: 0.9699

Loss on validation set:0.09878523978423327 Accuracy on validation set: 0.9699
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 22s 372us/step - loss: 0.1883 - acc: 0.9411 - val_loss: 0.0967 - val_acc: 0.9694

Loss on validation set:0.09673928351644427 Accuracy on validation set: 0.9694
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 21s 354us/step - loss: 0.1855 - acc: 0.9432 - val_loss: 0.1127 - val_acc: 0.9637

Loss on va

<b> 5J)</b> Build a convolutional network with the following structure:

<ul>
<li> A convolutional layer with 32 filters of size 3 × 3, with a ReLU activation
<li> A max pooling layer with size 2 × 2
<li> A convolutional layer with 64 filters of size 3 × 3, with ReLU activation
<li> A max pooling layer with size 2 × 2
<li> A flatten layer
<li> A fully connected layer with 128 neurons, with ReLU activation
<li> A dropout layer with drop probability 0.5
<li> A fully-connected layer with 10 neurons with softmax
</ul>
Train it on MNIST for one epoch, using <code>run_keras_cnn_mnist</code>.  What is the accuracy on the validation set?

If you have time to run the training for more epochs, try it, you should see improvement.


In [38]:
# uses scaled train/validation
# Conv1D, Conv2D, Dense, Dropout, Flatten, MaxPooling2D

layers = [Conv2D(input_shape=(28, 28, 1), filters=32, kernel_size=3, activation='relu'),
          MaxPooling2D(pool_size=(2,2)),
          Conv2D(filters=32, kernel_size=3, activation='relu'),
          MaxPooling2D(pool_size=(2,2)),
          Flatten(), 
          Dense(units=128, activation='relu'),
          Dropout(rate=0.5),
          Dense(units=10,  activation='softmax')    
]  # Your code

run_keras_cnn_mnist(train, validation, layers, 1, split=0.1, verbose=True, trials=5)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 59s 979us/step - loss: 0.2476 - acc: 0.9231 - val_loss: 0.0543 - val_acc: 0.9830

Loss on validation set:0.054268948423699476 Accuracy on validation set: 0.983
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 61s 1ms/step - loss: 0.2207 - acc: 0.9326 - val_loss: 0.0549 - val_acc: 0.9812

Loss on validation set:0.05485752918715589 Accuracy on validation set: 0.9812
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 61s 1ms/step - loss: 0.2304 - acc: 0.9294 - val_loss: 0.0493 - val_acc: 0.9852

Loss on validation set:0.049320278084510935 Accuracy on validation set: 0.9852
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============

<b> 5K)</b> Now, let's compare the performance of a fully connected model and a CNN on data where the characters have been shifted randomly so that they are no longer centered.  

You can build such a data set by calling: <code>train_20, validation_20 = get_MNIST_data(shift=20)</code>. Remember to scale it appropriately.

<b>Note that each image is now 48x48, so you will need to change your layer definitions</b>.
Run your two-hidden-layer FC architecture from above (problem 5I) on this data and then run the CNN architecture from above (problem 5J), both for one epoch. Report your results.


In [46]:
train_20, validation_20 = get_MNIST_data(shift=20) # Your code (fill in the shift)
m = np.max(train_20[0][0])
print(m)

# Scale the images
train_20 = (train_20[0]/m, train_20[1])  # Your code
validation_20 = (validation_20[0]/m, validation_20[1])  # Your code


255.0


In [47]:
train_20[0].shape

(60000, 48, 48)

In [48]:

layers_fc = [Dense(input_dim=48*48, units=512, activation='relu'), 
           Dense(units=256, activation='relu'), 
           Dense(units=10, activation="softmax")] 

run_keras_fc_mnist(train_20, validation_20, layers_fc, 1, split=0.1, verbose=True, trials=5)



Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 50s 841us/step - loss: 0.7392 - acc: 0.7573 - val_loss: 0.3884 - val_acc: 0.8763

Loss on validation set:0.3884438019275665 Accuracy on validation set: 0.8763
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 49s 814us/step - loss: 0.7463 - acc: 0.7542 - val_loss: 0.3963 - val_acc: 0.8709

Loss on validation set:0.39629622364044187 Accuracy on validation set: 0.8709
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 49s 813us/step - loss: 0.7443 - acc: 0.7546 - val_loss: 0.3756 - val_acc: 0.8807

Loss on validation set:0.375582682967186 Accuracy on validation set: 0.8807
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 48s 808us/step - loss: 0.7404 - acc: 0.7558 - val_loss: 0.3895 - val_acc: 0.8724

Loss on valid

In [49]:
layers_cnn = [Conv2D(input_shape=(48, 48, 1), filters=32, kernel_size=3, activation='relu'),
          MaxPooling2D(pool_size=(2,2)),
          Conv2D(filters=32, kernel_size=3, activation='relu'),
          MaxPooling2D(pool_size=(2,2)),
          Flatten(), 
          Dense(units=128, activation='relu'),
          Dropout(rate=0.5),
          Dense(units=10,  activation='softmax')    
]  # Your code  

run_keras_cnn_mnist(train_20, validation_20, layers_cnn, 1, split=0.1, verbose=True, trials=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 167s 3ms/step - loss: 0.7502 - acc: 0.7498 - val_loss: 0.1933 - val_acc: 0.9420

Loss on validation set:0.19327147540748119 Accuracy on validation set: 0.942
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 175s 3ms/step - loss: 0.7450 - acc: 0.7492 - val_loss: 0.1943 - val_acc: 0.9419

Loss on validation set:0.19434951719939708 Accuracy on validation set: 0.9419
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 174s 3ms/step - loss: 0.7204 - acc: 0.7575 - val_loss: 0.1574 - val_acc: 0.9555

Loss on validation set:0.15740453015714884 Accuracy on validation set: 0.9555
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 174s 3ms/step - loss: 0.7876 - acc: 0.7335 - val_loss: 0.1827 - val_acc: 0.9496

Loss on validat

<b> 5L)</b> Some possible conclusions. <a href="https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week8/week8_homework/">Refer to Catsoop.</a>

In [0]:
# 5B
train, validation = get_MNIST_data()
run_keras_fc_mnist(train, validation, [
    Dense(input_dim=28*28, units=10, activation="softmax")
], 1, split=0.1, verbose=False, trials=5)

#5C Answer
train, validation = get_MNIST_data()
run_keras_fc_mnist(train, validation, [
    Dense(input_dim=28*28, units=10, activation="softmax",
        kernel_initializer=VarianceScaling(
            scale=0.001, mode='fan_in', distribution='normal', seed=None
        )
    )
], 1, split=0.1, verbose=False, trials=5)

# 5D answer
train, validation = get_MNIST_data()
train = train[0] / 255, train[1]
validation = validation[0] / 255, validation[1]
run_keras_fc_mnist(train, validation, [
    Dense(input_dim=28*28, units=10, activation="softmax")
], 1, split=0.1, verbose=False, trials=5)

# 5F answer
train, validation = get_MNIST_data()
train = train[0] / 255, train[1]
validation = validation[0] / 255, validation[1]
run_keras_fc_mnist(train, validation, [
    Dense(input_dim=28*28, units=10, activation="softmax")
], 5, split=0.1, verbose=False, trials=5)
run_keras_fc_mnist(train, validation, [
    Dense(input_dim=28*28, units=10, activation="softmax")
], 10, split=0.1, verbose=False, trials=5)
run_keras_fc_mnist(train, validation, [
    Dense(input_dim=28*28, units=10, activation="softmax")
], 15, split=0.1, verbose=False, trials=5)

# 5H answers
train, validation = get_MNIST_data()
train = train[0] / 255, train[1]
validation = validation[0] / 255, validation[1]
run_keras_fc_mnist(train, validation, [
    Dense(input_dim=28*28, units=128, activation="relu"), Dense(units=10, activation="softmax")
], 1, split=0.1, verbose=False, trials=5)
run_keras_fc_mnist(train, validation, [
    Dense(input_dim=28*28, units=256, activation="relu"), Dense(units=10, activation="softmax")
], 1, split=0.1, verbose=False, trials=5)
run_keras_fc_mnist(train, validation, [
    Dense(input_dim=28*28, units=512, activation="relu"), Dense(units=10, activation="softmax")
], 1, split=0.1, verbose=False, trials=5)
run_keras_fc_mnist(train, validation, [
    Dense(input_dim=28*28, units=1024, activation="relu"), Dense(units=10, activation="softmax")
], 1, split=0.1, verbose=False, trials=5)

# 5I 
train, validation = get_MNIST_data()
train = train[0] / 255, train[1]
validation = validation[0] / 255, validation[1]
run_keras_fc_mnist(train, validation, [
    Dense(input_dim=28*28, units=512, activation="relu"),
    Dense(units=256, activation="relu"),
    Dense(units=10, activation="softmax")
], 1, split=0.1, verbose=False, trials=5)

# 5J
train, validation = get_MNIST_data()
train = train[0] / 255, train[1]
validation = validation[0] / 255, validation[1]
run_keras_cnn_mnist(train, validation, [
    Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(units=128, activation="relu"),
    Dropout(0.5),
    Dense(units=10, activation="softmax")
], 1, split=0.1, verbose=False, trials=5)

# 5K


train_20, validation_20 = get_MNIST_data(shift=20)
train_20 = train_20[0] / 255, train_20[1]
validation_20 = validation_20[0] / 255, validation_20[1]
run_keras_fc_mnist(train_20, validation_20, [
    Dense(input_dim=48*48, units=512, activation="relu"),
    Dense(units=256, activation="relu"),
    Dense(units=10, activation="softmax")
], 1, split=0.1, verbose=False, trials=5)
run_keras_cnn_mnist(train_20, validation_20, [
    Conv2D(32, (3, 3), activation="relu", input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(units=128, activation="relu"),
    Dropout(0.5),
    Dense(units=10, activation="softmax")
], 1, split=0.1, verbose=False, trials=5)
